# Workstation Heartbeat

This example shows how in a separate thread from the Term the HeartBeat can be queried with Python. The HeartBeat is queried in a separate thread once a second.

The HeartBeat represents how many milliseconds it has been since the Term has received something from the Thales.

In [4]:
import sys
from thales_remote.connection import ThalesRemoteConnection
from thales_remote.script_wrapper import PotentiostatMode,ThalesRemoteScriptWrapper
import time
import threading

zenniumConnection = None
zahnerZennium = None
keepThreadRunning = True

# Watch Thread
The following function is used as a thread, in which the HearBeat is queried once a second. The HeartBeat time varies, for example, if EIS is measured at low frequencies, then this time is increased.

With getTermIsActive() it is checked with timeout whether the Term software still answers, since beside Thales also the Term could have crashed.

In [5]:

def watchThreadFunction():
    global zenniumConnection
    global zahnerZennium
    global keepThreadRunning
    
    while keepThreadRunning:
        time.sleep(1)
        active = zahnerZennium.getTermIsActive()
        print("active state: " + str(active))
        if active:
            print("beat count: " + str(zahnerZennium.getWorkstationHeartBeat()))

# Main Program Sequence

The following is the main program flow, alongside which the Watch Thread runs. Here we simply measure something as an example.

In [ ]:
if __name__ == "__main__":
    zenniumConnection = ThalesRemoteConnection()
    zenniumConnection.connectToTerm("localhost", "ScriptRemote")
    
    zahnerZennium = ThalesRemoteScriptWrapper(zenniumConnection)
    zahnerZennium.forceThalesIntoRemoteScript()
    
    zahnerZennium.calibrateOffsets()

Now the watch thread is started in the main process, which runs alongside.

In [7]:
    testThread = threading.Thread(target=watchThreadFunction)
    testThread.start()
    print("heartbeat thread started")

heartbeat thread started


In the following, an impedance spectrum is simply measured as an example.

In [8]:
    zahnerZennium.setPotentiostatMode(PotentiostatMode.POTMODE_POTENTIOSTATIC)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setPotential(0)
    zahnerZennium.setLowerFrequencyLimit(500)
    zahnerZennium.setStartFrequency(1000)
    zahnerZennium.setUpperFrequencyLimit(10000)
    zahnerZennium.setLowerNumberOfPeriods(5)
    zahnerZennium.setLowerStepsPerDecade(2)
    zahnerZennium.setUpperNumberOfPeriods(20)
    zahnerZennium.setUpperStepsPerDecade(20)
    zahnerZennium.setScanDirection("startToMax")
    zahnerZennium.setScanStrategy("single")

    zahnerZennium.enablePotentiostat()
    
    zahnerZennium.setFrequency(1)
    zahnerZennium.setAmplitude(10e-3)
    zahnerZennium.setNumberOfPeriods(3)
    
    print("measurement start")
    zahnerZennium.measureEIS()
    print("measurement end")
    
    zahnerZennium.disablePotentiostat()

measurement start
active state: True
beat count: 422
active state: True
beat count: 312
active state: True
beat count: 859
active state: True
beat count: 1875
active state: True
beat count: 2891
active state: True
beat count: 781
active state: True
beat count: 360
active state: True
beat count: 1375
active state: True
beat count: 860
active state: True
beat count: 437
active state: True
beat count: 31
active state: True
beat count: 1047
active state: True
beat count: 656
active state: True
beat count: 187
active state: True
beat count: 1203
active state: True
beat count: 765
active state: True
beat count: 328
active state: True
beat count: 1343
active state: True
beat count: 922
active state: True
beat count: 515
active state: True
beat count: 110
active state: True
beat count: 1125
active state: True
beat count: 672
active state: True
beat count: 218
active state: True
beat count: 1234
active state: True
beat count: 797
active state: True
beat count: 375
active state: True
beat count:

'OK\r'

Close the watch thread and wait until it is closed.

In [9]:
    print("thread kill")
    keepThreadRunning = False
    testThread.join()
    print("thread killed")

active state: True
thread kill
beat count: 0
thread killed


# Disconnect

After the measurements are completed, the device switches back to the main potentiostat and the connection to the term is disconnected.

In [10]:
    zenniumConnection.disconnectFromTerm()
    print("finish")

finish
